In [1]:
import pandas as pd
from datetime import timedelta
import re
from typing import Dict, Tuple
import numpy as np
import pickle

In [2]:
all_vacancies_df = pd.read_csv('data/all_vacancies.csv')
print(all_vacancies_df.shape)
all_vacancies_df.head(2)

(4208, 19)


,text,name,salary,experience,employment_type,company_name,address,skills,uid,datetime,query_name,link,employment_day,employment_workhours,publish_date,city,min_salary,max_salary,city_rating
0,!!! Просьба при отклике на вакансию указывать ...,Руководитель отдела баз данных и отчетности,з/п не указана,3–6 лет,"Полная занятость, полный день",ООО Деметра Трейдинг,NaN,"['PostgreSQL', 'Python', 'VBA', 'MS SQL', 'Dat...",69816019,Вакансия опубликована 19 сентября 2022 в Москве,Data analyst,https://hh.ru/vacancy/69816019?from=vacancy_se...,Полная занятость,Полный день,2022-09-19,Москва,NaN,NaN,0.548012
1,!!! Просьба при отклике на вакансию указывать ...,Руководитель отдела баз данных и отчетности,з/п не указана,3–6 лет,"Полная занятость, полный день",ООО Деметра Трейдинг,"Краснодар, улица Красных Партизан, 200","['PostgreSQL', 'Python', 'VBA', 'MS SQL', 'Dat...",68928909,Вакансия опубликована 23 сентября 2022 в Красн...,Data engineer,https://hh.ru/vacancy/68928909?from=vacancy_se...,Полная занятость,Полный день,2022-09-23,Краснодар,NaN,NaN,0.013503


In [3]:
def create_profession_map_file(path: str):
    profession_map_df = pd.DataFrame(columns=['query_name', 'profession', 'domen'])

    map = [
        {'query_name':'Аналитик данных', 'profession':'Аналитик данных', 'domen':'Data Science'},
        {'query_name':'Bi analyst', 'profession':'Bi аналитик', 'domen':'Data Science'},
        {'query_name':'Системный инженер', 'profession':'Системный инженер', 'domen':''},
        {'query_name':'Data analyst', 'profession':'Инженер данных', 'domen':'Data Science'},
        {'query_name':'Дата инженер', 'profession':'Инженер данных', 'domen':'Data Science'},
        {'query_name':'Аналитик', 'profession':'Аналитик', 'domen':'Data Science'},
        {'query_name':'Bi аналитик', 'profession':'Bi аналитик', 'domen':'Data Science'},
        {'query_name':'Machine learning', 'profession':'ML инженер', 'domen':'Data Science'},
        {'query_name':'Data engineer', 'profession':'Инженер данных', 'domen':'Data Science'},
        {'query_name':'Главный бухгалтер', 'profession':'Главный бухгалтер', 'domen':''},
        {'query_name':'Data scientist', 'profession':'Data scientist', 'domen':'Data Science'},
        {'query_name':'Системный аналитик', 'profession':'Системный аналитик', 'domen':''}
    ]

    for row in map:
        profession_map_df = profession_map_df.append(row, ignore_index=True)

    profession_map_df.to_csv(path, index=False)

create_profession_map_file('data/profession_map.csv')
profession_map_df = pd.read_csv('data/profession_map.csv')
profession_map_df


/var/folders/3c/_r_wzsdn5h3fr5qwhjx2hb440000gn/T/ipykernel_24983/1212321181.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  profession_map_df = profession_map_df.append(row, ignore_index=True)
/var/folders/3c/_r_wzsdn5h3fr5qwhjx2hb440000gn/T/ipykernel_24983/1212321181.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  profession_map_df = profession_map_df.append(row, ignore_index=True)
/var/folders/3c/_r_wzsdn5h3fr5qwhjx2hb440000gn/T/ipykernel_24983/1212321181.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  profession_map_df = profession_map_df.append(row, ignore_index=True)
/var/folders/3c/_r_wzsdn5h3fr5qwhjx2hb440000gn/T/ipykernel_24983/1212321181.py:20: FutureWarning: The frame.append method is deprecated and

,query_name,profession,domen
0,Аналитик данных,Аналитик данных,Data Science
1,Bi analyst,Bi аналитик,Data Science
2,Системный инженер,Системный инженер,NaN
3,Data analyst,Инженер данных,Data Science
4,Дата инженер,Инженер данных,Data Science
5,Аналитик,Аналитик,Data Science
6,Bi аналитик,Bi аналитик,Data Science
7,Machine learning,ML инженер,Data Science
8,Data engineer,Инженер данных,Data Science
9,Главный бухгалтер,Главный бухгалтер,NaN


In [4]:
def get_slice(df: pd.DataFrame, profession_map_df: pd.DataFrame, domen: str = 'Data Science', 
              last_days: int = 14) -> pd.DataFrame:
    """Return DataFrame filtred by Domen and Timestamp
        
        Parameters
        ----------
        df: pd.DataFrame :
            Dataframe with columns ['query_name', 'publish_date']
        profession_map_df: pd.DataFrame :
            DataFrame with profesion mapping, columns ['query_name', 'profession', 'domen']
        domen: str:
            Filter condition
        last_days: int
            Slice interval

        Returns
        -------
        pd.DataFrame : 
            Slice of df
    """

    df = df.dropna(subset=['publish_date'])
    df.publish_date = pd.to_datetime(df.publish_date)
    
    from_date = df.publish_date.max() - timedelta(days=last_days)
    df = df[df.publish_date > from_date]

    queries = set(profession_map_df[profession_map_df['domen'] == domen].query_name.to_list())

    return df[df['query_name'].isin(queries)]

df = get_slice(all_vacancies_df, profession_map_df)
df.to_csv('data/features/slice_df.csv')
df.shape

/var/folders/3c/_r_wzsdn5h3fr5qwhjx2hb440000gn/T/ipykernel_24983/1109455570.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.publish_date = pd.to_datetime(df.publish_date)


(1998, 19)

In [10]:
# skill processing

def get_all_skills(df: pd.DataFrame) -> Tuple[Dict[str, float], pd.DataFrame]:
    """Returns Skills dictionary
        
        Parameters
        ----------
        df: pd.DataFrame :
            Dataframe with columns ['skills'] (! will be added some colums without copy !)
        profession_map_df: pd.DataFrame :
            DataFrame with profesion mapping, columns ['query_name', 'profession', 'domen']
        domen: str:
            Filter condition
        last_days: int
            Slice interval

        Returns
        -------
        Tuple[Dict[str, float], pd.DataFrame] : 
            Skills dictionary. Key is skill. Value is normalized frequency
            Updated data frame with new column 'skill_set' that contain set of skills
    """
    r = re.compile('\\\\|////|,')
    df['skill_set'] = df.skills.apply(lambda s : {x.strip(" '") for x in re.split(r, s.strip('[]'))} - {''})
    all_skills = set().union(*df.skill_set.to_list())

    skill_counts = {}
    rows_count = df.shape[0]
    for skill in all_skills:
        skill_counts[skill] = df.skill_set.apply(lambda x: skill in x).sum() / rows_count

    return skill_counts, df


def skills_preprocess(skills: Dict[str, float]) -> Tuple[Dict[int, str], Dict[str, int]]:
    """Skills preprocess:
        - discarding case, spec-sumbols
        - <some more later... (rule-based, spell-shecking, etc)>
        - choosing the best skill name based frequency of skills in group

        Parameters
        ----------
        skills: Dict[str, float] :
            Dict skill name to frequency

        Returns
        -------
        Tuple[Dict[int, str], Dict[str, int]] :
            Tuple of:
                - dict index to corrected skill name
                - dist if original skill name to index
     
    """

    #! map may be moved to config
    #!!! ещй имеет разделят такие скилы: Excel/GoogleSheets
    #!!! обхединить более крпные группы тоже ?
    correction_map = [
        ['1 С Бухгалтерия', '1С: Бухгалтерия'],
        ['1C ERP', '1C ЕРП', '1С: ERP'],
        ['1C', '1C', '1С.', '1c', '1с', '1C', '1С', 'Знание 1С'],
        ['AirfFlow', 'Airflow', 'AirFlow', 'Apache Airflow'],
        ['Apache Superset', 'Superset'],
        ['ERP', 'ERP)'],
        ['ETL\\\\ELT', 'ETL'],
        ['English', 'English for IT', 'Английский', 'Английский язык',
            'Английский язык (Intermediate)', 'Английский язык (уровень Intermediate)', 'Английский — B1 — Средний',
            'Английский язык (Upper-Intermediate)', 'Английский — B2 — Средне-продвинутый',
            'Знание английского языка', 'xa0— A1 — Начальный', 'xa0— A2 — Элементарный',
            'xa0— B1 — Средний', 'xa0— B2 — Средне-продвинутый', 'xa0— C1 — Продвинутый',
            'xa0— C2 — В совершенстве'],
        ['A/B тестирование', 'A/B тесты', 'a/b-experiments', 'A/B testing Power BI', 'Проведение тестирований',
            'А/Б тестирование', 'А/В тесты', 'АБ-тесты'],
        ['ABC analysis', 'ABC-анализ'],
        ['Регресcионное тестирование', 'Регрессионное тестирование'],
        ['1C: Предприятие', '1С: Предприятие'],
        ['Cassandra', 'Apache Cassandra'],
        ['Apache Hadoop', 'Hadoop', 'hadoop'],
        ['Apache Hive', 'Hive', 'hive'],
        ['Apache Kafka', 'Kafka'],
        ['Apache Spark', 'Spark'],
        ['AppMetrica', 'AppMetrika'],

        ['ML', 'ML Engineering', 'ML-методы', 'Машинное обучение', 'Machine Learning'],
        ['Analytics', 'Analysis', 'Analytical skills', 'Аналитик', 'Аналитика', 'analyst',
            'Аналитическая работа', 'Аналитические навыки', 'Аналитический склад ума',
            'Аналитическое мышление', 'Умение анализировать', 
            'Умение анализировать и интерпретировать данные'],
        ['Data Analysis', 'Data Analyst', 'Data Analytics'],
        ['Data Mining', 'Data Mining: Business Analytica', 'Data Mining: SPSS', 'Data Mining: Statistica'],
        ['Data инженер', 'Data Engineer', 'Data Engineering'],
        ['Dashboard', 'Dashboards'],
        ['Data Science', 'Data Scientist'],
        ['Машинное обучение', 'Machine Learning'],
        ['OLAP', 'OLAP-кубы', 'Olap (online analytical processing)', 'База данных: Olap'],
        ['ORACLE', 'Oracle Database', 'База данных: Oracle'],
        ['Вайлдберис', 'Вайлдберриз'],
        ['ERP', 'ERP Systems'],
        ['MS SQL', 'MS SQL Server'],
        ['MS Visio', 'Visio'],
        ['MS Access', 'Access', 'Microsoft Access'],
        ['MS Axapta', 'Axapta'],
        ['MS Azure', 'Azure', 'Microsoft Azure', 'Microsoft Azure Service Fabric'],
        ['MS Visual Studio', 'Microsoft Visual Studio'],
        ['MS Excel', 'MS Excel (уровень: программирование на VBA)', 'MC Excel', 'Знание Excel',
            'Знание Excel', 'Работа в Excel', 'Опытный пользователь Excel и MS Office',
            'Продвинутый пользователь Excel', 'Продвинутый пользователь MS Excel'],
        ['MS Internet', 'MS Internet Explorer'],
        ['MS Office', 'MS Office (Word', 'знание MS Office'],
        ['MS Outlook', 'Outlook)', 'Outlook'],
        ['MS Power BI', 'отчетность в Power BI', 'Power BI', 'Power Bl', 'PowerBI'],
        ['Power Query', 'Power qwery'],
        ['MS PowerPoint', 'PowerPoint'],
        ['MS SQL', 'MS SQL Server'],
        ['MS Visio', 'Visio'],
        ['Знание Word', 'умение работать с программами Word', 'MS Word', 'Word'],
        ['Интернет маркетинг', 'Интеренет-маркетинг', 'Internet Marketing'],
        ['Инженер данных', 'дата инженер', 'Data Engineer', 'Data Engineering', 'Data инженер'],
        ['Django', 'Django Framework'],
        ['IT', 'ИТ'],
        ['Логика', 'Логическое мышление'],
        ['Аналитик данных', 'Аналитика данных', 'Data Analysis', 'Data Analyst', 'Data Analytics'],
        ['Data Science', 'Data Scientist'],
        ['Нейронные сети', 'нейросети', 'Neural Networks', 'neural nets'],
        ['Озон', 'Ozon'],
        ['Опытный пользователь ПК', 'Опытный пользовать ПК'],
        ['Ответственность', 'Ответственность и пунктуальность', 'Ответственный подход к работе'],
        ['Работа с соц.сетями', 'Работа с социальными сетями'],
        ['Разработка тестов', 'юнит-тестирование', 'Unit Testing'],
        ['С#', 'C#'],
        ['C++', 'C/C++'],
        ['CRM системы', 'CRM-система'],
        ['B2B Marketing', 'B2B маркетинг'],
        ['Arenadata', 'Arenadata DB (Greenplum)'],
        ['BPMN', 'BPMN 2.0'],
        ['Уверенный пользователь MS Office', 'Уверенный пользователь Microsoft Excel и 1С8', 'Уверенный пользователь ПК'],
        ['Умение анализировать', 'Умение анализировать и интерпретировать данные'],
        ['Умение организовать работу', 'Умение организовывать работу'],
        ['Яндекс.Метрика', 'Яндекс метрика'],
        ['методологии', 'методология'],
        ['техническое зрение', 'Computer Vision', 'машинное зрение'],
        ['руководство командой разработчиков', 'Leadership Skills', 'Лидерские качества', 
            'Лидерство', 'Team lead', 'Team management', 'Управление командой', 'Teamleading',
            'Teambuilding'],
        ['Product Owner', 'руководство проектом', 'Управление продуктом'],
        ['Project Manager', 'Project management', 'Управление проектами'],
        ['BI система', 'BI системы'],
        ['Jupiter', 'Jupyter Notebook'],
        ['Photon', 'Phyton', 'Python'],
        ['PostreSQL', 'PostgreSQL'],
        ['R', 'R-программирование'],
        ['SEO', 'SEO оптимизация', 'SEO-оптимизация карточек товара'],
        ['SIEM', 'SIEM systems'],
        ['SQL Server Reporting Services', 'SSRS'],
        ['SQL запросы', 'SQL программист', 'SQL.', 'SQL', 'Transact-SQL', 'Базовые знания SQL'],
        ['SWOT', 'SWOT анализ'],
        ['Software architect', 'Software architecture'],
        ['Spark', 'Spark SQL', 'Spark Streaming'],
        ['Spring', 'Spring Framework'],
        ['Web Design', 'Web-дизайн'],
        ['image processing', 'Обработка изображений'],
        ['.NET Framework', '.NET'],
        ['UX/UI', 'UI/UX'],
        ['Use Case Analysis', 'Use cases'],
        ['GOlang', 'go'],
        ['Google Cloud', 'Google Cloud Platform'],
        ['Google Analytics', 'Google Analytics 360 and GA4'],
        ['Google Data Studio', 'Google Data Studiо'],
        ['Google Sheet', 'Google Sheets', 'Google Spreadsheets', 'Google tables', 'Google таблицы'],
        ['React', 'React.js', 'ReactJS'],
        ['Tableau', 'Tableu', 'Tabular Editor'],
        ['sklearn', 'Scikit-learn'],
        ['Высокая энергичность', 'Высокая энергичность и инициативность'],
        ['Гибкость', 'Гибкость мышления'],
        ['Грамотная письменная речь', 'Грамотная речь', 'Грамотная речь и письмо',
            'Грамотная устная и письменная речь', 'Грамотность'],
        ['Dashboards', 'Дашборды']


    ]

    def simplify(s):
        for x in correction_map:
            if s.lower() in [y.lower() for y in x]:
                s = x[0]
                break

        return s.lower().replace(' ', '').replace('-', '').replace('/','').replace(':','')

    new_index  = 0
    index_to_corrected = {}
    original_to_index = {}
    simplified_to_index = {}
    for skill in skills.keys():
        simplified_skill = simplify(skill)
        index = simplified_to_index.get(simplified_skill, new_index)
        original_to_index[skill] = index

        if index == new_index:
            simplified_to_index[simplified_skill] = index
            index_to_corrected[index] = skill
            new_index += 1
        else:
            skill_aliases = [k for k, v in original_to_index.items() if v == index]
            if skill not in skill_aliases:
                skill_aliases += [skill]
            best_name = skill_aliases[np.array([skills[x] for x in skill_aliases]).argmax()]
            index_to_corrected[index] = best_name

    return index_to_corrected, original_to_index


all_skills, df = get_all_skills(df)
skill_index_to_corrected, skill_original_to_index = skills_preprocess(all_skills)
with open('data/features/skills.txt', 'w') as f:
    f.writelines([x + '\n' for x in sorted(skill_index_to_corrected.values())])
with open('data/features/skill_index_to_corrected.pkl', 'wb') as f:
    pickle.dump(skill_index_to_corrected, f)
with open('data/features/skill_original_to_index.pkl', 'wb') as f:
    pickle.dump(skill_original_to_index, f)

print('all skills =', len(set().union(*df.skill_set.to_list())))
print('skills without case =', len(set([x.lower() for x in set().union(*df.skill_set.to_list())])))
print('skills after processing =', len(skill_index_to_corrected))

all skills = 1968
skills without case = 1869
skills after processing = 1673


In [6]:
def professions_preprocess(df: pd.DataFrame, map: Dict[str, str]) -> Tuple[Dict[int, str], Dict[str, int]]:
    """Skills preprocess:
    Parameters
        ----------
        df: pd.DataFrame :
            Dataframe with query_name column
        map: Dict[str, str] :
            Map query_name to profession name

    Returns
        -------
        Tuple[Dict[int, str], Dict[str, int]] :
            Tuple of:
                - dict index to profession name
                - dist if original query_name to index
    """

    index_to_corrected = {}
    original_to_index = {}
    prof_to_index = {}

    new_index = 0
    for q in df.query_name.unique():
        prof = map.get(q, q)
        index = prof_to_index.get(prof, new_index)
        prof_to_index[prof] = index
        index_to_corrected[index] = prof
        original_to_index[q] = index
        if new_index == index:
            new_index += 1

    return index_to_corrected, original_to_index

map = dict(zip(profession_map_df.query_name, profession_map_df.profession))
prof_index_to_corrected, prof_original_to_index = professions_preprocess(df, map)
with open('data/features/prof_index_to_corrected.pkl', 'wb') as f:
    pickle.dump(prof_index_to_corrected, f)
with open('data/features/prof_original_to_index.pkl', 'wb') as f:
    pickle.dump(prof_original_to_index, f)

In [40]:
# skills table, extra features

def create_skill_df(index_to_corrected: Dict[int, str]) -> pd.DataFrame:
    """Returns dataframe of skills"""

    skill_df = pd.DataFrame()
    skill_df['skill'] = index_to_corrected.values()
    skill_df['skill_id'] = index_to_corrected.keys()
    return skill_df

def add_features_to_skill(skill_df: pd.DataFrame, df: pd.DataFrame, 
                          original_to_index: Dict[str, int], min_vacancies = 10) -> pd.DataFrame:
    """Add calculated features to skills dataframe"""

    salary_q25 = {}
    salary_q50 = {}
    salary_q75 = {}
    skill_freq = {}

    df = df[['skill_set', 'min_salary', 'max_salary']]
    df['skill_ind_set'] = df.skill_set.apply(lambda x: set([original_to_index[s] for s in x]))

    for s in skill_df.skill_id:
        s_df = df[df.skill_ind_set.apply(lambda x: s in x)]
        skill_freq[s] = s_df.shape[0] / df.shape[0]
        s_df = s_df[~s_df.min_salary.isna() & ~s_df.max_salary.isna()]
        if s_df.shape[0] > min_vacancies:
            salaries = (s_df.min_salary + s_df.max_salary) / 2
            salary_q25[s] = round(salaries.quantile(0.25))
            salary_q50[s] = round(salaries.quantile(0.50))
            salary_q75[s] = round(salaries.quantile(0.75))

    skill_df['salary_q25'] = skill_df.skill_id.apply(lambda x: salary_q25.get(x, None))
    skill_df['salary_q50'] = skill_df.skill_id.apply(lambda x: salary_q50.get(x, None))
    skill_df['salary_q75'] = skill_df.skill_id.apply(lambda x: salary_q75.get(x, None))
    skill_df['frequency'] = skill_df.skill_id.apply(lambda x: skill_freq.get(x, None))

    return skill_df

skill_df = create_skill_df(skill_index_to_corrected)
skill_df = add_features_to_skill(skill_df, df, skill_original_to_index)
skill_df.sort_values(by='salary_q50', ascending=False).head(20)

/var/folders/3c/_r_wzsdn5h3fr5qwhjx2hb440000gn/T/ipykernel_15229/2870903962.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['skill_ind_set'] = df.skill_set.apply(lambda x: set([original_to_index[s] for s in x]))


,skill,skill_id,salary_q25,salary_q50,salary_q75,frequency
1226,Java,1226,250000.0,304500.0,396419.0,0.035536
855,ETL,855,170500.0,250000.0,260000.0,0.047548
1421,Kafka,1421,175000.0,240000.0,308588.0,0.022523
946,xa0— C1 — Продвинутый,946,147900.0,234300.0,264221.0,0.020521
261,Docker,261,100000.0,233919.0,310000.0,0.019019
361,Git,361,143750.0,221250.0,305000.0,0.039540
1390,PyTorch,1390,150000.0,217500.0,304500.0,0.018519
559,ClickHouse,559,190000.0,200502.0,298588.0,0.021522
1316,MySQL,1316,135000.0,200000.0,225000.0,0.015015
951,JavaScript,951,143750.0,200000.0,262500.0,0.016517


In [43]:
skill_df.sort_values(by='frequency', ascending=False).head(10)

,skill,skill_id,salary_q25,salary_q50,salary_q75,frequency,popular_profession_id
264,SQL,264,87000.0,132750.0,199800.0,0.389890,0
1256,Python,1256,120000.0,160000.0,250000.0,0.239239,0
765,Аналитическое мышление,765,50000.0,72040.0,113655.0,0.207708,1
1027,Английский,1027,80000.0,130000.0,199933.0,0.191692,0
1203,Анализ данных,1203,50000.0,75500.0,120000.0,0.182182,2
614,MS PowerPoint,614,46734.0,70000.0,90000.0,0.168669,1
730,Power BI,730,70189.0,95000.0,130500.0,0.115616,3
1760,Работа с большим объемом информации,1760,45468.0,65250.0,88750.0,0.115115,2
9,Бизнес-анализ,9,73125.0,100000.0,150000.0,0.083584,1
91,MS SQL,91,85250.0,130250.0,175000.0,0.074074,2


In [10]:
# profesions table, extra features

def create_prof_df(index_to_corrected: Dict[int, str]) -> pd.DataFrame:
    """Returns dataframe of professions"""

    prof_df = pd.DataFrame()
    prof_df['prof_name'] = index_to_corrected.values()
    prof_df['prof_id'] = index_to_corrected.keys()
    return prof_df

def add_features_to_prof(prof_df: pd.DataFrame, df: pd.DataFrame, profession_map_df: pd.DataFrame) -> pd.DataFrame:
    """Add calculated features to profecions dataframe"""

    salary_q25 = {}
    salary_q50 = {}
    salary_q75 = {}
    prof_freq = {}

    for p in prof_df.prof_name:
        queries = set(profession_map_df[profession_map_df['profession'] == p].query_name.to_list())
        p_df = df[df['query_name'].isin(queries)]
        prof_freq[p] = p_df.shape[0] / df.shape[0]
        p_df = p_df[~p_df.min_salary.isna() & ~p_df.max_salary.isna()]
        if p_df.shape[0] > 0:
            salaries = (p_df.min_salary + p_df.max_salary) / 2
            salary_q25[p] = round(salaries.quantile(0.25))
            salary_q50[p] = round(salaries.quantile(0.50))
            salary_q75[p] = round(salaries.quantile(0.75))

    prof_df['salary_q25'] = prof_df.prof_name.apply(lambda x: salary_q25.get(x, None))
    prof_df['salary_q50'] = prof_df.prof_name.apply(lambda x: salary_q50.get(x, None))
    prof_df['salary_q75'] = prof_df.prof_name.apply(lambda x: salary_q75.get(x, None))
    prof_df['frequency'] = prof_df.prof_name.apply(lambda x: prof_freq.get(x, None))

    return prof_df

prof_df = create_prof_df(prof_index_to_corrected)
prof_df = add_features_to_prof(prof_df, df, profession_map_df)
prof_df.sort_values(by='salary_q50', ascending=False)

,prof_name,prof_id,salary_q25,salary_q50,salary_q75,frequency
4,ML инженер,4,143950,230462,302627,0.053053
5,Data scientist,5,150000,208750,291632,0.029029
0,Инженер данных,0,92500,155000,230731,0.260260
3,Bi аналитик,3,60975,87250,135827,0.196196
2,Аналитик данных,2,60000,80000,120900,0.234234
1,Аналитик,1,50000,70035,104400,0.227227


Есть три способа сохрания матрицы А:
- просто добавить колонки в датафреймы (удобно для хранения и может фильтрации)
- desnse matrix (удобно для T-NSE)
- sparse matrix (удобно для ALS)

Попробую разные способы... если тормозов не будет, то почему бы и не dataframe..

А если делать всякие tf-idf? вручную на матрице? с этой точки зрения матрица удобней, а фильтровать можно по индексам из датафрейма

In [44]:
# Skill-Profesion relationship matrix will stored in numpy matrix
# This type has severall benefits:
#   - acceptable size (max 10000x20 elements of float = 1.6Mb)
#   - fast processing such as normalization, sums, etc
#   - fast filtring via index

def get_skill_prof_matrix(df: pd.DataFrame, skill_original_to_index: Dict[str, int],
                           prof_original_to_index: Dict[str, int]) -> np.array:
    """Returns Skill-Profesion relationship matrix"""

    matrix = np.zeros((len(skill_index_to_corrected), len(prof_index_to_corrected)))
    for row in df[['query_name', 'skill_set']].to_numpy():
        prof_id = prof_original_to_index[row[0]]
        for s in row[1]:
            skill_id = skill_original_to_index[s]
            matrix[skill_id, prof_id] += 1

    return matrix

matrix = get_skill_prof_matrix(df, skill_original_to_index, prof_original_to_index)
with open('data/features/matrix.pkl', 'wb') as f:
    pickle.dump(matrix, f)

In [65]:
matrix.sum(axis=0).shape

(6,)

In [71]:
def add_popular_profession_feature(skill_df: pd.DataFrame, prof_df: pd.DataFrame, 
                                   matrix: np.array, prof_index_to_corrected: Dict[int, str]) -> pd.DataFrame:
    """Add weighed most popular profession to each skill"""
    prof_frequency = prof_df.sort_values(by='prof_id').frequency.to_numpy()
    skill_df['popular_profession_id'] = (matrix / prof_frequency).argmax(axis=1)
    skill_df['popular_profession_name'] = skill_df.popular_profession_id.apply(lambda x: prof_index_to_corrected[x])

    matrix_colsum = matrix.sum(axis=0)
    for k, v in prof_index_to_corrected.items():
        skill_df[v] = skill_df.apply(lambda x: \
                        matrix[x.skill_id, k] / matrix_colsum[k], axis='columns')

def add_popular_skills_feature(prof_df: pd.DataFrame, matrix: np.array,
                                   skill_index_to_corrected: Dict[int, str]) -> pd.DataFrame:
    """Add weighed most popular profession to each skill"""
    prof_df['popular_skills'] = prof_df.prof_id.apply(lambda i: 
                      ', '.join([skill_index_to_corrected[x] for x in matrix[:,i].argsort()[::-1]][:10]))

add_popular_profession_feature(skill_df, prof_df, matrix, prof_index_to_corrected)
skill_df.to_csv('data/features/skill_table.csv', index=False)

add_popular_skills_feature(prof_df, matrix, skill_index_to_corrected)
prof_df.to_csv('data/features/prof_table.csv', index=False)

display(skill_df.sort_values(by='frequency', ascending=False).head(15))
display(prof_df.head(15))


,skill,skill_id,salary_q25,salary_q50,salary_q75,frequency,popular_profession_id,popular_profession_name,frequency_in_popular_profession,Инженер данных,Аналитик,Аналитик данных,Bi аналитик,ML инженер,Data scientist
264,SQL,264,87000.0,132750.0,199800.0,0.389890,2,Аналитик данных,0.062145,0.062145,0.027855,0.068147,0.065770,0.038208,0.066489
1256,Python,1256,120000.0,160000.0,250000.0,0.239239,5,Data scientist,0.050527,0.050527,0.006190,0.029510,0.026163,0.075099,0.119681
765,Аналитическое мышление,765,50000.0,72040.0,113655.0,0.207708,1,Аналитик,0.043330,0.012699,0.043330,0.039854,0.032340,0.006588,0.007979
1027,Английский,1027,80000.0,130000.0,199933.0,0.191692,4,ML инженер,0.047555,0.047555,0.024451,0.027989,0.028706,0.055336,0.026596
1203,Анализ данных,1203,50000.0,75500.0,120000.0,0.182182,2,Аналитик данных,0.044722,0.012429,0.026927,0.044722,0.026890,0.007905,0.010638
614,MS PowerPoint,614,46734.0,70000.0,90000.0,0.168669,1,Аналитик,0.036212,0.006755,0.036212,0.034378,0.027980,0.002635,0.007979
730,Power BI,730,70189.0,95000.0,130500.0,0.115616,3,Bi аналитик,0.043605,0.005404,0.010214,0.019471,0.043605,0.002635,0.002660
1760,Работа с большим объемом информации,1760,45468.0,65250.0,88750.0,0.115115,1,Аналитик,0.028293,0.001891,0.028165,0.028293,0.013445,0.001318,0.002660
9,Бизнес-анализ,9,73125.0,100000.0,150000.0,0.083584,1,Аналитик,0.018261,0.004053,0.018261,0.017037,0.012718,0.001318,0.002660
91,MS SQL,91,85250.0,130250.0,175000.0,0.074074,3,Bi аналитик,0.016428,0.006755,0.005881,0.016428,0.017078,0.001318,0.005319


,prof_name,prof_id,salary_q25,salary_q50,salary_q75,frequency,popular_skills
0,Инженер данных,0,92500,155000,230731,0.260260,"SQL, Python, Английский, Java, PostgreSQL, Lin..."
1,Аналитик,1,50000,70035,104400,0.227227,"Аналитическое мышление, MS PowerPoint, Работа ..."
2,Аналитик данных,2,60000,80000,120900,0.234234,"SQL, Анализ данных, Аналитическое мышление, MS..."
3,Bi аналитик,3,60975,87250,135827,0.196196,"SQL, Power BI, Аналитическое мышление, Английс..."
4,ML инженер,4,143950,230462,302627,0.053053,"Python, Machine Learning, Английский, SQL, PyT..."
5,Data scientist,5,150000,208750,291632,0.029029,"Python, SQL, Machine Learning, PyTorch, Англий..."
